In [1]:
import numpy as np
import sys
import pandas as pd
import copy

# Request generation function

In [2]:
def create_mars_request(reqpath, request,dryrun=True):
    
    """
    Generates a MARS request at location reqpath based on the request dictionary.
    """
    
    req = 'retrieve,\n'
    
    for key,value in request.items():
        req += '{}={},\n'.format(key,value)
        
    if dryrun:
        
        print('saving request below to {}:\n\n{}'.format(reqpath,req))
        return 
        
    with open(reqpath,'w') as f:

        f.write(req)

# Set global variables

In [3]:
## create list of experiment details
exp_list = pd.DataFrame(columns=['id','ptype','fctype','fcdate'])

exps = ['b2nn','b2no','b2nq','b2nr','b2ns','b2nt']
ptype = ['pi','incr']*3
fcdate = ['2022-02-14']*2+['2022-02-10']*2+['2022-02-16']*2

for i in range(6):
    exp_list.loc[i*2] = [exps[i],ptype[i],'cf',fcdate[i]]
    exp_list.loc[i*2+1] = [exps[i],ptype[i],'pf',fcdate[i]]

In [4]:
exp_list

,id,ptype,fctype,fcdate
0,b2nn,pi,cf,2022-02-14
1,b2nn,pi,pf,2022-02-14
2,b2no,incr,cf,2022-02-14
3,b2no,incr,pf,2022-02-14
4,b2nq,pi,cf,2022-02-10
5,b2nq,pi,pf,2022-02-10
6,b2nr,incr,cf,2022-02-10
7,b2nr,incr,pf,2022-02-10
8,b2ns,pi,cf,2022-02-16
9,b2ns,pi,pf,2022-02-16


In [5]:
outdir = '/gf3/predict2/AWH012_LEACH_NASTORM/DATA/'

### Parameter lists

In [ ]:
param_longnames = ["10 metre U wind component", "10 metre V wind component", "10 metre wind gust since previous post-processing", "100 metre U wind component", "100 metre V wind component", "2 metre dewpoint temperature", "2 metre temperature", "Albedo", "Average potential temperature in the upper 300m", "Average salinity in the upper 300m", "Averaged total lightning flash density in the last 6 hours", "Boundary layer dissipation", "Boundary layer height", "Charnock", "Convective available potential energy", "Convective inhibition", "Convective precipitation", "Convective rain rate", "Depth of 20C isotherm", "Eastward sea water velocity", "Eastward turbulent surface stress", "Evaporation", "Forecast surface roughness", "Geopotential", "Gravity wave dissipation", "High cloud cover", "Instantaneous 10 metre wind gust", "Instantaneous large-scale surface precipitation fraction", "Instantaneous total lightning flash density", "K index", "Land-sea mask", "Large scale rain rate", "Large-scale precipitation", "Low cloud cover", "Maximum CAPE in the last 6 hours", "Maximum CAPES in the last 6 hours", "Maximum temperature at 2 metres since previous post-processing", "Maximum total precipitation rate since previous post-processing", "Mean sea level pressure", "Medium cloud cover", "Minimum temperature at 2 metres since previous post-processing", "Mixed layer depth", "Northward sea water velocity", "Northward turbulent surface stress", "Potential evaporation", "Precipitation type", "Runoff", "Sea ice area fraction", "Sea surface height", "Sea surface temperature", "Sea water practical salinity", "Sea-ice thickness", "Skin temperature", "Surface latent heat flux", "Surface roughness", "Surface sensible heat flux", "Surface thermal radiation downwards", "Top net solar radiation", "Top net thermal radiation", "Total cloud cover", "Total column vertically-integrated water vapour", "Total column water", "Total precipitation", "Total totals index", "Visibility"]



param_df = pd.Dataframe(columns = )



# 1 deg global

In [23]:
# sfc - exp
basepath = outdir+'{exptype}/{ptype}/GLO100/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

param = '34.128/49.128/136.128/137.128/151.128/164.128/226.228/228.128/228035/228036'

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for i,S in exp_list.iterrows():
    
    exp,ptype,fctype,fcdate = S.values
                
    fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
    steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
    exp = explist.loc[exptype,ptype]

    reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

    req = 'retrieve,\n'
    req += 'class=uk,\n'
    req += 'date='+fcdate+',\n'
    req += 'expver='+exp+',\n'
    req += 'levtype=sfc,\n'
    if fctype=='pf':
        req += 'number='+number+',\n'
    req += 'param='+param+',\n'
    req += 'step='+steps+',\n'
    req += 'stream=enfo,\n'
    req += 'time=00:00:00,\n'
    req += 'type='+fctype+',\n'
    req += 'format=netcdf,\n'
    req += 'grid=1.0/1.0,\n'
    req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

#     with open(reqpath,'w') as f:

#         f.write(req)

# # just have to change prev pp precip max to 6 hrly precip max & 10fg
# param = '34.128/123.128/136.128/137.128/151.128/164.128/224.228/228.128/228035/228036'

# # sfc - ENS
# exptype = 'MED-R/ENS'
# for fctype in ['pf','cf']:
#     for fcdate in ['2022-02-07']:

#         fclength = (pd.to_datetime('2022-02-22') - pd.to_datetime(fcdate)).days * 24
#         steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
#         exp = '1'

#         reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

#         req = 'retrieve,\n'
#         req += 'class=od,\n'
#         req += 'date='+fcdate+',\n'
#         req += 'expver='+exp+',\n'
#         req += 'levtype=sfc,\n'
#         if fctype=='pf':
#             req += 'number='+number+',\n'
#         req += 'param='+param+',\n'
#         req += 'step='+steps+',\n'
#         req += 'stream=enfo,\n'
#         req += 'time=00:00:00,\n'
#         req += 'type='+fctype+',\n'
#         req += 'format=netcdf,\n'
#         req += 'grid=1.0/1.0,\n'
#         req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
#         with open(reqpath,'w') as f:

#             f.write(req)

# sfc - large ensembles
# fcdate = '2021-06-18'
# number = '/'.join(np.arange(1,251).astype(str))
# for exp,expptype in LElist.items():
#     for fctype in ['pf','cf']:

#         fclength = (pd.to_datetime('2021-07-02') - pd.to_datetime(fcdate)).days * 24
#         steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
#         exptype,ptype = expptype.split('_')

#         reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

#         req = 'retrieve,\n'
#         req += 'class=uk,\n'
#         req += 'date='+fcdate+',\n'
#         req += 'expver='+exp+',\n'
#         req += 'levtype=sfc,\n'
#         if fctype=='pf':
#             req += 'number='+number+',\n'
#         req += 'param='+param+',\n'
#         req += 'step='+steps+',\n'
#         req += 'stream=enfo,\n'
#         req += 'time=00:00:00,\n'
#         req += 'type='+fctype+',\n'
#         req += 'format=netcdf,\n'
#         req += 'grid=1.0/1.0,\n'
#         req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

#         with open(reqpath,'w') as f:

#             f.write(req)

In [24]:
# pl - exp
basepath = outdir+'{exptype}/{ptype}/glo100/pl/{fctype}/req_files/{exp}_{fcdate}.req'

params = ['129.128/130.128','129.128','129.128/131/132']
levels= ['850','500','250']

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for exptype in ['cmip6']:
    for ptype in ['pi','incr']:
        for fctype in ['pf','cf']:
            for fcdate in ['2021-06-22']:
                
                fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
                steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
                exp = explist.loc[exptype,ptype]
                
                reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)
                
                req = 'retrieve,\n'
                req += 'class=uk,\n'
                req += 'date='+fcdate+',\n'
                req += 'expver='+exp+',\n'
                req += 'levelist='+levels[0]+',\n'
                req += 'levtype=pl,\n'
                if fctype=='pf':
                    req += 'number='+number+',\n'
                req += 'param='+params[0]+',\n'
                req += 'step='+steps+',\n'
                req += 'stream=enfo,\n'
                req += 'time=00:00:00,\n'
                req += 'type='+fctype+',\n'
                req += 'format=netcdf,\n'
                req += 'grid=1.0/1.0,\n'
                req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
                
                for i in np.arange(1,3):
                    req += 'retrieve,\n'
                    req += 'levelist='+levels[i]+',\n'
                    req += 'param='+params[i]+'\n\n'
                
#                 with open(reqpath,'w') as f:

#                     f.write(req)
                
# pl - ENS
exptype = 'ENS'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-20']:

        fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levelist='+levels[0]+',\n'
        req += 'levtype=pl,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+params[0]+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
        for i in np.arange(1,3):
            req += 'retrieve,\n'
            req += 'levelist='+levels[i]+',\n'
            req += 'param='+params[i]+'\n\n'
        
#         with open(reqpath,'w') as f:

#             f.write(req)

# sfc - large ensembles
fcdate = '2021-06-18'
number = '/'.join(np.arange(1,251).astype(str))
for exp,expptype in LElist.items():
    for fctype in ['pf','cf']:

        fclength = (pd.to_datetime('2021-07-02') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
        exptype,ptype = expptype.split('_')

        reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=uk,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levelist='+levels[0]+',\n'
        req += 'levtype=pl,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+params[0]+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

        for i in np.arange(1,3):
            req += 'retrieve,\n'
            req += 'levelist='+levels[i]+',\n'
            req += 'param='+params[i]+'\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

# 0.25 deg EU

In [21]:
# sfc - exp
basepath = outdir+'{exptype}/{ptype}/EU025/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

param = '34.128/49.128/136.128/137.128/151.128/164.128/226.228/228.128' # for ENS
param = '34.128/49.128/136.128/137.128/151.128/164.128/226.228/228.128' # for EXP

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
# step_ens = np.arange(6,360,6)
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

# for i,S in exp_list.iterrows():
    
#     param = '34.128/49.128/136.128/137.128/151.128/164.128/226.228/228.128/165.128/166.128/246.228/247.228' # for EXP
    
#     exp,ptype,fctype,fcdate = S.values
    
#     exptype='MED-R/EXP'
                
#     fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
#     steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])

#     reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

#     req = 'retrieve,\n'
#     req += 'class=uk,\n'
#     req += 'date='+fcdate+',\n'
#     req += 'expver='+exp+',\n'
#     req += 'levtype=sfc,\n'
#     if fctype=='pf':
#         req += 'number='+number+',\n'
#     req += 'param='+param+',\n'
#     req += 'step='+steps+',\n'
#     req += 'stream=enfo,\n'
#     req += 'time=00:00:00,\n'
#     req += 'type='+fctype+',\n'
#     req += 'format=netcdf,\n'
#     req += 'grid=0.25/0.25,\n'
#     req += 'area=E,\n'
#     req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
    
#     ## 6-hrly data
#     steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength] if x%6==0][1:])
#     param='228035/228036'
#     req += 'retrieve,\n'
#     req += 'param='+param+',\n'
#     req += 'step='+steps+'\n\n'

#     with open(reqpath,'w') as f:

#         f.write(req)
                
# sfc - ENS
# exptype = 'MED-R/ENS'
# for fctype in ['pf','cf']:
#     for fcdate in ['2022-02-{:02d}'.format(x) for x in [10,14,16]]:
        
#         param = '34.128/49.128/136.128/137.128/151.128/164.128/226.228/228.128/165.128/166.128/246.228/247.228'

#         fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
#         steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
#         exp = '1'

#         reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

#         req = 'retrieve,\n'
#         req += 'class=od,\n'
#         req += 'date='+fcdate+',\n'
#         req += 'expver='+exp+',\n'
#         req += 'levtype=sfc,\n'
#         if fctype=='pf':
#             req += 'number='+number+',\n'
#         req += 'param='+param+',\n'
#         req += 'step='+steps+',\n'
#         req += 'stream=enfo,\n'
#         req += 'time=00:00:00,\n'
#         req += 'type='+fctype+',\n'
#         req += 'format=netcdf,\n'
#         req += 'grid=0.25/0.25,\n'
#         req += 'area=E,\n'
#         req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
#         ## 6-hrly data
#         steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength] if x%6==0][1:])
#         param='228035/228036'
#         req += 'retrieve,\n'
#         req += 'param='+param+',\n'
#         req += 'step='+steps+'\n\n'
        
#         with open(reqpath,'w') as f:

#             f.write(req)

In [18]:
# pl - exp
basepath = outdir+'{exptype}/{ptype}/EU025/pl/{fctype}/req_files/{exp}_{fcdate}.req'

param = '129.128/131/132/138.128'

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for i,S in exp_list.iterrows():
    
    exp,ptype,fctype,fcdate = S.values
    
    exptype='MED-R/EXP'
                
    fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
    steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])

    reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

    req = 'retrieve,\n'
    req += 'class=uk,\n'
    req += 'date='+fcdate+',\n'
    req += 'expver='+exp+',\n'
    req += 'levelist=850/700/500,\n'
    req += 'levtype=pl,\n'
    if fctype=='pf':
        req += 'number='+number+',\n'
    req += 'param='+param+',\n'
    req += 'step='+steps+',\n'
    req += 'stream=enfo,\n'
    req += 'time=00:00:00,\n'
    req += 'type='+fctype+',\n'
    req += 'format=netcdf,\n'
    req += 'grid=0.25/0.25,\n'
    req += 'area=E,\n'
    req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

    with open(reqpath,'w') as f:

        f.write(req)
                
# pl - ENS
exptype = 'MED-R/ENS'
for fctype in ['pf','cf']:
    for fcdate in ['2022-02-{:02d}'.format(x) for x in [10,14,16]]:
        
        fclength = (pd.to_datetime('2022-02-20') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levtype=pl,\n'
        req += 'levelist=850/700/500,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=E,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
        with open(reqpath,'w') as f:

            f.write(req)

In [22]:
pd.to_datetime('2022-03-03')-pd.to_datetime('2022-01-27')

Timedelta('35 days 00:00:00')

# 2.5 deg global all pressure levels

In [14]:
# pl - exp
basepath = outdir+'{exptype}/{ptype}/glo250/pl/{fctype}/req_files/{exp}_{fcdate}.req'

param = '129.128/130.128/131/132/133.128/135.128/155.128/157.128'
level = '10/50/100/200/250/500/700/850/925/1000'

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for exptype in ['cmip6']:
    for ptype in ['pi','incr']:
        for fctype in ['pf','cf']:
            for fcdate in ['2021-06-22']:
                
                fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
                steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
                exp = explist.loc[exptype,ptype]
                
                reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)
                
                req = 'retrieve,\n'
                req += 'class=uk,\n'
                req += 'date='+fcdate+',\n'
                req += 'expver='+exp+',\n'
                req += 'levelist='+level+',\n'
                req += 'levtype=pl,\n'
                if fctype=='pf':
                    req += 'number='+number+',\n'
                req += 'param='+param+',\n'
                req += 'step='+steps+',\n'
                req += 'stream=enfo,\n'
                req += 'time=00:00:00,\n'
                req += 'type='+fctype+',\n'
                req += 'format=netcdf,\n'
                req += 'grid=2.5/2.5,\n'
                req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
                
#                 with open(reqpath,'w') as f:

#                     f.write(req)
                
# pl - ENS
exptype = 'ENS'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-20']:

        fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levelist='+level+',\n'
        req += 'levtype=pl,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=2.5/2.5,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
#         with open(reqpath,'w') as f:

#             f.write(req)

            
# pl - large ensembles
fcdate = '2021-06-18'
number = '/'.join(np.arange(1,251).astype(str))
for exp,expptype in LElist.items():
    for fctype in ['pf','cf']:
        for numbers in np.arange(5):
            number = '/'.join(np.arange(numbers*50+1,(numbers+1)*50+1).astype(str))
            pfname = '_'+number.split('/')[0]+'-'+number.split('/')[-1]

            fclength = (pd.to_datetime('2021-07-02') - pd.to_datetime(fcdate)).days * 24
            steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
            exptype,ptype = expptype.split('_')

            req = 'retrieve,\n'
            req += 'class=uk,\n'
            req += 'date='+fcdate+',\n'
            req += 'expver='+exp+',\n'
            req += 'levelist='+level+',\n'
            req += 'levtype=pl,\n'
            if fctype=='pf':
                req += 'number='+number+',\n'
            else:
                pfname = ''
            req += 'param='+param+',\n'
            req += 'step='+steps+',\n'
            req += 'stream=enfo,\n'
            req += 'time=00:00:00,\n'
            req += 'type='+fctype+',\n'
            req += 'format=netcdf,\n'
            req += 'grid=2.5/2.5,\n'
            req += 'target="'+exp+'_'+fcdate+pfname+'.nc"\n\n'
            
            reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate+pfname)

            with open(reqpath,'w') as f:

                f.write(req)

In [37]:
# sfc - exp
basepath = outdir+'{exptype}/{ptype}/glo250/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

param = ''

step_ens = np.array(np.arange(90).tolist() + np.arange(90,144,3).tolist() + np.arange(144,360,6).tolist())
step_exp = np.array(np.arange(0,360,3).tolist())

number = '/'.join(np.arange(1,51).astype(str))

for exptype in ['cmip6']:
    for ptype in ['pi','incr']:
        for fctype in ['pf','cf']:
            for fcdate in ['2021-06-22']:
                
                fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
                steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
                exp = explist.loc[exptype,ptype]
                
                reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)
                
                req = 'retrieve,\n'
                req += 'class=uk,\n'
                req += 'date='+fcdate+',\n'
                req += 'expver='+exp+',\n'
                req += 'levtype=sfc,\n'
                if fctype=='pf':
                    req += 'number='+number+',\n'
                req += 'param='+param+',\n'
                req += 'step='+steps+',\n'
                req += 'stream=enfo,\n'
                req += 'time=00:00:00,\n'
                req += 'type='+fctype+',\n'
                req += 'format=netcdf,\n'
                req += 'grid=2.5/2.5,\n'
                req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
                
#                 with open(reqpath,'w') as f:

#                     f.write(req)
                
# sfc - ENS
exptype = 'ENS'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-20']:

        fclength = (pd.to_datetime('2021-07-01') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_ens[step_ens<=fclength]])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levtype=sfc,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'
        
#         with open(reqpath,'w') as f:

#             f.write(req)


# sfc - large ensembles
fcdate = '2021-06-18'
number = '/'.join(np.arange(1,251).astype(str))
for exp,expptype in LElist.items():
    for fctype in ['pf','cf']:

        fclength = (pd.to_datetime('2021-07-02') - pd.to_datetime(fcdate)).days * 24
        steps = '/'.join([str(x) for x in step_exp[step_exp<=fclength]])
        exptype,ptype = expptype.split('_')

        reqpath = basepath.format(exptype=exptype,ptype=ptype,fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=uk,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'levtype=sfc,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfo,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'format=netcdf,\n'
        req += 'grid=2.5/2.5,\n'
        req += 'target="'+exp+'_'+fcdate+'.nc"\n\n'

#         with open(reqpath,'w') as f:

#             f.write(req)

# M-climate data

Dates required:

- 2021-07-08
- 2021-07-05
- 2021-07-01
- 2021-06-28
- 2021-06-24
- 2021-06-21
- 2021-06-17
- 2021-06-14
- 2021-06-10
- 2021-06-07
- 2021-06-03

## pnw025

In [21]:
# sfc - M-climate
basepath = outdir+'{exptype}/{ptype}/pnw025/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

param = '39.128/137.128/151.128/164.128/167.128/182.128/201.128/228.128'

step_mclim = np.array(np.arange(0,313,6).tolist())

number = '/'.join(np.arange(1,11).astype(str))

exptype='m-clim'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-'+x for x in ['03','07','10','14','17','21','24','28']]+['2021-07-'+x for x in ['01','05','08']]:

        steps = '/'.join([str(x) for x in step_mclim])
        hdate = '/'.join([str(x)+fcdate[4:] for x in np.arange(2001,2021)])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'hdate='+hdate+',\n'
        req += 'levtype=sfc,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfh,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=70/-150/30/-100,\n'
        req += 'target="'+exp+'_'+fcdate+'.grib"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

In [5]:
# pl - M-climate
basepath = outdir+'{exptype}/{ptype}/pnw025/pl/{fctype}/req_files/{exp}_{fcdate}.req'

param = '130.128'

step_mclim = np.array(np.arange(0,313,6).tolist())

number = '/'.join(np.arange(1,11).astype(str))

exptype='m-clim'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-'+x for x in ['03','07','10','14','17','21','24','28']]+['2021-07-'+x for x in ['01','05','08']]:

        steps = '/'.join([str(x) for x in step_mclim])
        hdate = '/'.join([str(x)+fcdate[4:] for x in np.arange(2001,2021)])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'hdate='+hdate+',\n'
        req += 'levelist=850,\n'
        req += 'levtype=pl,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfh,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'grid=0.25/0.25,\n'
        req += 'area=70/-150/30/-100,\n'
        req += 'target="'+exp+'_'+fcdate+'.grib"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

## glo100

In [22]:
# sfc - M-climate
basepath = outdir+'{exptype}/{ptype}/glo100/sfc/{fctype}/req_files/{exp}_{fcdate}.req'

param = '151.128/167.128/228.128'

step_mclim = np.array(np.arange(0,313,6).tolist())

number = '/'.join(np.arange(1,11).astype(str))

exptype='m-clim'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-'+x for x in ['03','07','10','14','17','21','24','28']]+['2021-07-'+x for x in ['01','05','08']]:

        steps = '/'.join([str(x) for x in step_mclim])
        hdate = '/'.join([str(x)+fcdate[4:] for x in np.arange(2001,2021)])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'hdate='+hdate+',\n'
        req += 'levtype=sfc,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfh,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcdate+'.grib"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

In [23]:
# pl - M-climate
basepath = outdir+'{exptype}/{ptype}/glo100/pl/{fctype}/req_files/{exp}_{fcdate}.req'

param = '129.128'

step_mclim = np.array(np.arange(0,313,6).tolist())

number = '/'.join(np.arange(1,11).astype(str))

exptype='m-clim'
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-'+x for x in ['03','07','10','14','17','21','24','28']]+['2021-07-'+x for x in ['01','05','08']]:

        steps = '/'.join([str(x) for x in step_mclim])
        hdate = '/'.join([str(x)+fcdate[4:] for x in np.arange(2001,2021)])
        exp = '1'

        reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate)

        req = 'retrieve,\n'
        req += 'class=od,\n'
        req += 'date='+fcdate+',\n'
        req += 'expver='+exp+',\n'
        req += 'hdate='+hdate+',\n'
        req += 'levelist=500,\n'
        req += 'levtype=pl,\n'
        if fctype=='pf':
            req += 'number='+number+',\n'
        req += 'param='+param+',\n'
        req += 'step='+steps+',\n'
        req += 'stream=enfh,\n'
        req += 'time=00:00:00,\n'
        req += 'type='+fctype+',\n'
        req += 'grid=1.0/1.0,\n'
        req += 'target="'+exp+'_'+fcdate+'.grib"\n\n'

        with open(reqpath,'w') as f:

            f.write(req)

## glo250
Split into each parameter, split u/v into each number.

In [16]:
# pl - M-climate
basepath = outdir+'{exptype}/{ptype}/glo250/pl/{fctype}/req_files/{exp}_{fcdate}.req'

step_mclim = np.array(np.arange(0,313,12).tolist())

number = '/'.join(np.arange(1,11).astype(str))

exptype='m-clim'

param_dict = dict(zip(['z','t','u','v','q','w','d'],['129.128','130.128','131','132','133.128','135.128','155.128']))

## get each param individually
for fctype in ['pf','cf']:
    for fcdate in ['2021-06-'+x for x in ['03','07','10','14','17','21','24','28']]+['2021-07-'+x for x in ['01','05','08']]:

        for pname,param in param_dict.items():
            
            if pname in ['u','v'] and fctype=='pf':
                
                for numbers in number.split('/'):
                
                    steps = '/'.join([str(x) for x in step_mclim])
                    hdate = '/'.join([str(x)+fcdate[4:] for x in np.arange(2001,2021)])
                    exp = '1'

                    reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate+'_'+pname+'_'+numbers)

                    req = 'retrieve,\n'
                    req += 'class=od,\n'
                    req += 'date='+fcdate+',\n'
                    req += 'expver='+exp+',\n'
                    req += 'hdate='+hdate+',\n'
                    req += 'levelist=10/50/100/200/300/400/500/700/850/925/1000,\n'
                    req += 'levtype=pl,\n'
                    req += 'number='+numbers+',\n'
                    req += 'param='+param+',\n'
                    req += 'step='+steps+',\n'
                    req += 'stream=enfh,\n'
                    req += 'time=00:00:00,\n'
                    req += 'type='+fctype+',\n'
                    req += 'grid=2.5/2.5,\n'
                    req += 'target="'+exp+'_'+fcdate+'_'+pname+'_'+numbers+'.grib"\n\n'

                    with open(reqpath,'w') as f:

                        f.write(req)
                
            else:
        
                steps = '/'.join([str(x) for x in step_mclim])
                hdate = '/'.join([str(x)+fcdate[4:] for x in np.arange(2001,2021)])
                exp = '1'

                reqpath = basepath.format(exptype=exptype,ptype='',fctype=fctype,exp=exp,fcdate=fcdate+'_'+pname)

                req = 'retrieve,\n'
                req += 'class=od,\n'
                req += 'date='+fcdate+',\n'
                req += 'expver='+exp+',\n'
                req += 'hdate='+hdate+',\n'
                req += 'levelist=10/50/100/200/300/400/500/700/850/925/1000,\n'
                req += 'levtype=pl,\n'
                if fctype=='pf':
                    req += 'number='+number+',\n'
                req += 'param='+param+',\n'
                req += 'step='+steps+',\n'
                req += 'stream=enfh,\n'
                req += 'time=00:00:00,\n'
                req += 'type='+fctype+',\n'
                req += 'grid=2.5/2.5,\n'
                req += 'target="'+exp+'_'+fcdate+'_'+pname+'.grib"\n\n'

                with open(reqpath,'w') as f:

                    f.write(req)